In [ ]:
!pip install turicreate 
import turicreate as tc
from turicreate import SFrame

In [ ]:
sf=tc.SFrame('AllProductReviews.csv')


In [ ]:
sf.head(2)

# **adding  word count vector**

In [ ]:
sf['word_count']=tc.text_analytics.count_words(sf['ReviewBody'])

In [ ]:
sf.head(10)


ReviewTitle,ReviewBody,ReviewStar,Product
Honest review of an edmmusic lover\n ...,No doubt it has a greatbass and to a great ...,3,boAt Rockerz 255
Unreliable earphones withhigh cost\n ...,"This earphones areunreliable, i bought it ...",1,boAt Rockerz 255
Really good anddurable.\n ...,"i bought itfor 999,Ipurchased it second t ...",4,boAt Rockerz 255
stopped working in just14 days\n ...,Its sound quality isadorable. overall it was ...,1,boAt Rockerz 255
Just Awesome WirelessHeadphone under ...,Its Awesome... Good soundquality & 8-9 hrs bat ...,5,boAt Rockerz 255
Charging port notworking\n ...,"After 11 days, thecharging port isn't ...",1,boAt Rockerz 255
Loved the color and sound!!\n ...,Sound-4.5/5Buildquality-4.5/5Battery ...,4,boAt Rockerz 255
Great sound with worstmic.\n ...,After one month usagereviewing this .Pros-1- ...,3,boAt Rockerz 255
Battery life is goodenough to keep going all ...,I can’t get enough ofthis earphone! The bass ...,5,boAt Rockerz 255
Very very bad Durabity\n,The product durability is1 month..I just lost my ...,1,boAt Rockerz 255


In [ ]:
sf['Product'].show()

Materializing SArray

In [ ]:
boat_review=sf[sf['Product']=='boAt Rockerz 255']
len(boat_review)

In [ ]:
jbl_review=sf[sf['Product']=='JBL T110BT']
jbl_review['ReviewStar'].mean()


In [ ]:
boat_review['ReviewStar'].mean()
# dir(boat_review['ReviewStar'])

In [ ]:
boat_review.head(2)


In [ ]:
# jbl_review.head(1)

ReviewTitle,ReviewBody,ReviewStar,Product,word_count,sentiments
Substandard audioquality\n ...,Please do not buy this.Wired earphones worth ...,2,JBL T110BT,"{'within': 1.0,'product': 1.0, 'the': ...",0


# **building sentimental classifier**

In [ ]:
boat_review=boat_review[boat_review['ReviewStar']!=3]
# neglected all reviews

In [ ]:
# jbl_review=jbl_review[jbl_review['ReviewStar']!=3]

In [ ]:
sf['sentiments']=sf['ReviewStar']>=4
sf.head(1)



ReviewTitle,ReviewBody,ReviewStar,Product,word_count,sentiments
Honest review of an edmmusic lover\n ...,No doubt it has a greatbass and to a great ...,3,boAt Rockerz 255,"{'voice': 1.0,'complaining': 1.0, ...",0


# **training sentimental classifier**

In [ ]:
# split data
train_data,test_data=sf.random_split(.8,seed=0)

In [ ]:
sentimental_model=tc.classifier.create(train_data,target='sentiments',features=['word_count'],validation_set=test_data)

# **evaluate model**

In [ ]:
sentimental_model.evaluate(test_data,metric='confusion_matrix')
# sentimental_model.training_f1_score()


{'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |  732  |
 |      1       |        1        |  1584 |
 |      0       |        1        |  214  |
 |      1       |        0        |  246  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

In [ ]:
# sentimental_model.evaluate('accuracy')
sentimental_model.evaluate(test_data,metric='accuracy')

{'accuracy': 0.8342939481268011}

In [ ]:

# dir(sentimental_model)

# **applied learned model to undertsand sentiment of other**

In [ ]:
jbl_review['predicted_sentiment']=sentimental_model.predict(jbl_review,output_type='margin')

In [ ]:
jbl_review=jbl_review.sort('predicted_sentiment',ascending=True)
jbl_review.head(5)

Columns:
	ReviewTitle	str
	ReviewBody	str
	ReviewStar	int
	Product	str
	word_count	dict
	sentiments	int
	predicted_sentiment	float

Rows: 5

Data:
+-------------------------------+-------------------------------+------------+
|          ReviewTitle          |           ReviewBody          | ReviewStar |
+-------------------------------+-------------------------------+------------+
| Don't waste your money on ... | This is my personal experi... |     1      |
| Don't go through this mess... | Listening to your neighbou... |     1      |
|       Breach of faith\n       | Hey guysHighly not recomme... |     1      |
| Functional. Not best-in-cl... | Pros: Functional, does the... |     3      |
|    Serious Pairing issues\n   | there is a serious pairing... |     1      |
+-------------------------------+-------------------------------+------------+
+------------+-------------------------------+------------+---------------------+
|  Product   |           word_count          | sentiments | predicted_sentiment |
+------------+-------------------------------+------------+---------------------+
| JBL T110BT | {'7': 1.0, 's9': 1.0, '5t'... |     0      |  -32.56286719710787 |
| JBL T110BT | {'only': 1.0, 'whole': 1.0... |     0      | -28.898426519723827 |
| JBL T110BT | {'vain': 1.0, 'in': 1.0, '... |     0      |  -25.12892180319392 |
| JBL T110BT | {'elsewhere': 1.0, 'looks'... |     0      | -22.829534165544356 |
| JBL T110BT | {'now': 1.0, 'jbl': 1.0, '... |     0      |  -22.54918807849353 |
+------------+-------------------------------+------------+---------------------+
[5 rows x 7 columns]

In [ ]:
jbl_review.head(5)


In [ ]:
jbl_review[0]['ReviewBody']

In [ ]:
jbl_review[-1]['ReviewBody']